In [3]:
import matplotlib.pyplot as plt
import numpy as np
from custom_env.actions import Acciones
from ipywidgets import interact,interactive_output
from gymnasium import make
from stable_baselines3 import PPO
import ipywidgets
from IPython.display import clear_output


In [4]:
seed = 289980628190634006122751570777790489191
name = "./models/Example"
env_id = "RouterEnv-v0"

env = make(env_id, seed=seed)
seed: int = env.np_random_seed
model = PPO.load(name, print_system_info=True)

num_steps = 1
obs, _ = env.reset(seed=seed)
# Variables de información:

stats = []
rewards = []

# -------------------------

ant = env._get_info()
for episode in range(num_steps):

    done = False
    step_counter = 0
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, terminated, info = env.step(action)

        stats.append(info["Stats"])
        rewards.append(reward)

        done: bool = done or terminated

    env.reset(seed=seed)
    step_counter += 1

== CURRENT SYSTEM INFO ==
- OS: Windows-11-10.0.26100-SP0 10.0.26100
- Python: 3.12.6
- Stable-Baselines3: 2.4.1
- PyTorch: 2.5.1+cpu
- GPU Enabled: False
- Numpy: 1.26.4
- Cloudpickle: 3.1.0
- Gymnasium: 1.0.0

== SAVED MODEL SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.12.6
- Stable-Baselines3: 2.4.1
- PyTorch: 2.5.1+cpu
- GPU Enabled: False
- Numpy: 1.26.4
- Cloudpickle: 3.1.0
- Gymnasium: 1.0.0



In [5]:
# Definir la función de recompensa


def compute_reward(actual: float, descartados: int, action: Acciones) -> float:
    reward: float = 0.0
    c = 0.25
    c1=0.5
    if descartados > 0:
        if action == Acciones.PERMITIR:
            reward -= (descartados**2) * c1
        else:
            reward -= (descartados) * c
    else:
        reward += 1.0
    return reward


def create_plot(axes, fila, actual, valores, recompensas, accion: Acciones):
    col: int = Acciones.action_to_int(accion)
    axes[fila, col].plot(valores, recompensas, label='Reward')
    axes[fila, col].set_xlabel('Descartados')
    axes[fila, col].set_ylabel('Recompensa')
    axes[fila, col].set_title(f'{accion.name} - Reward - {actual:.2f}')
    axes[fila, col].legend()
    axes[fila, col].grid(True)


def plot_reward(descartados_range=8, num=3):
    actual_values = np.linspace(0, 1, num)
    descartados_values = np.arange(0, descartados_range + 1)
    # assert actual_values.shape == descartados_values.shape

    fig, axes = plt.subplots(num, 2, figsize=(10, num * 3))
    axes = np.atleast_2d(axes)  # Asegurarse de que sea una matriz 2D

    for i, actual in enumerate(actual_values):
        recompensas_permitir: list[float] = [compute_reward(
            actual, d, Acciones.PERMITIR) for d in descartados_values]
        recompensas_denegar: list[float] = [compute_reward(
            actual, d, Acciones.DENEGAR) for d in descartados_values]
        create_plot(axes, i, actual, descartados_values,
                    recompensas_permitir, Acciones.PERMITIR)
        create_plot(axes, i, actual, descartados_values,
                    recompensas_denegar, Acciones.DENEGAR)

    plt.tight_layout()
    plt.show()


# Crear la interactividad
interact(plot_reward, descartados_range=(1, 20, 1), num=(2, 10, 1))

interactive(children=(IntSlider(value=8, description='descartados_range', max=20, min=1), IntSlider(value=3, d…

<function __main__.plot_reward(descartados_range=8, num=3)>

In [6]:
from matplotlib import axes
from matplotlib.lines import Line2D


def re_ocu(actual: float, coef=55) -> float:
    reward = 0.0
    reward += (1-actual)*coef
    return reward


def re_desca(descartados: float, coef=1.0) -> float:
    reward = 0.0
    if descartados > 0:
        reward -= (descartados**2) * coef
    else:
        reward += 60.0
    return reward


def re_accion(action) -> float:
    reward = 0.0
    match action:
        case Acciones.PERMITIR:
            reward += 5.0
        case Acciones.DENEGAR:
            reward -= 2.0
        case _:
            reward += 0.0
    return reward


def plot_subplot(ax: axes._axes.Axes, datos_actual: list[float], recompensa_actual_in: list[float], datos_descartados: list[int], recompensa_descartados_in: list[float], accion: Acciones):

    recompensa_actual = list(map(lambda x: x + re_accion(accion), recompensa_actual_in))
    recompensa_descartados = list(map(lambda x: x + re_accion(accion), recompensa_descartados_in))


    line1: list[Line2D] = ax.plot(datos_actual, recompensa_actual,
                            label='Ocupación', color='blue')
    
    ax.set_xlabel('Ocupación')
    ax.set_ylabel('Recompensa')
    ax.spines['bottom'].set_color('blue')
    ax.xaxis.label.set_color('blue')
    ax.tick_params(axis='x', colors='blue')

    ax2: axes.Axes = ax.twiny()
    line2: Line2D = ax2.plot(
        datos_descartados, recompensa_descartados, label='Descartados', color='red')
    ax2.set_xlabel('Descartados')
    ax2.spines['top'].set_color('red')
    ax2.xaxis.label.set_color('red')
    ax2.tick_params(axis='x', colors='red')

    min_rew: float = min(recompensa_actual + recompensa_descartados)
    max_rew: float = max(recompensa_actual + recompensa_descartados)
    ax.set_ylim(min_rew, max_rew)
    ax2.set_ylim(min_rew, max_rew)

    lines: list[Line2D] = [*line1, *line2]
    labels: list[object] = [line.get_label() for line in lines]
    ax.legend(lines, labels)
    ax.set_title(accion.name)


def generate_subplots(axes, fila, datos_actual, datos_descartados, recompensa_actual, recompensa_descartados):
    plot_subplot(axes[fila, 0], datos_actual, recompensa_actual,
                 datos_descartados, recompensa_descartados, Acciones.PERMITIR)
    plot_subplot(axes[fila, 1], datos_actual, recompensa_actual,
                 datos_descartados, recompensa_descartados, Acciones.DENEGAR)


def generate_plot(coef_actual, coef_descartados, num_descartados=2):
    datos_actual = np.linspace(0, 1, num_descartados)
    datos_descartados = np.arange(0, num_descartados + 1)

    recompensa_actual = [re_ocu(a, coef=coef_actual) for a in datos_actual]
    recompensa_descartados = [re_desca(d, coef=coef_descartados) for d in datos_descartados]
    filas = 1
    fig, axes = plt.subplots(1, 2, figsize=(10, filas * 4))
    axes = np.atleast_2d(axes)

    generate_subplots(axes, 0, datos_actual, datos_descartados,
                      recompensa_actual, recompensa_descartados)

    plt.tight_layout()
    plt.show()


interact(generate_plot, coef_actual=(-100, 100, 5),
         coef_descartados=(1, 200, 5), num_descartados=(2, 10, 1))

interactive(children=(IntSlider(value=0, description='coef_actual', min=-100, step=5), IntSlider(value=96, des…

<function __main__.generate_plot(coef_actual, coef_descartados, num_descartados=2)>

In [7]:
from gymnasium.spaces import Box, Dict, Discrete

observation_space = Dict({
            "OcupacionCola": Box(low=0, high=1, dtype=np.float32),
            "Descartados": Box(low=0, high=np.inf, dtype=np.int64),  
        })
print(observation_space.values())
observation_space.sample()

ValuesView(Dict('Descartados': Box(0, 9223372036854775807, (1,), int64), 'OcupacionCola': Box(0.0, 1.0, (1,), float32)))


{'Descartados': array([0], dtype=int64),
 'OcupacionCola': array([0.17427889], dtype=float32)}

## Nuevo

In [8]:
import matplotlib.pyplot as plt
import numpy as np
from custom_env.actions import Acciones
from ipywidgets import interact,interactive_output
from gymnasium import make
from stable_baselines3 import PPO
import ipywidgets
from IPython.display import clear_output
from custom_env.router_env import reward

In [9]:
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt


clear_output(wait=True)


# Función para graficar la recompensa en función de c y c1


def plot_reward(c=0.25, c1=0.5, max_descartados=10):
    def local_reward(descartados: int,
                     action: Acciones,
                     c: float,
                     c2: float) -> float:
        return reward(descartados=descartados, action=action, ocu_actual=0.0, ocu_ant=0.0,c=c, c2=c2,c3=0.0)
    descartados_range = np.arange(0, max_descartados + 1, 1)

    rewards_permitir = [local_reward(
        d, Acciones.PERMITIR, c, c1) for d in descartados_range]
    rewards_bloquear = [local_reward(
        d, Acciones.DENEGAR, c, c1) for d in descartados_range]

    plt.figure(figsize=(8, 5))
    plt.plot(descartados_range, rewards_permitir,
             label="Acción: PERMITIR", marker="o", linestyle="--")
    plt.plot(descartados_range, rewards_bloquear,
             label="Acción: DENEGAR", marker="s", linestyle="-")
    plt.axhline(0, color="black", linewidth=1, linestyle="dotted")

    plt.xlabel("Descartados")
    plt.ylabel("Recompensa")
    plt.title("Impacto de c y c1 en la recompensa")
    plt.legend()
    plt.grid(True)
    plt.show()


# Crear widgets interactivos
interact(plot_reward,
         c=(0, 4, 0.05),
         c1=(0, 4, 0.05),
         max_descartados=(3, 20, 1),
         )

interactive(children=(FloatSlider(value=0.25, description='c', max=4.0, step=0.05), FloatSlider(value=0.5, des…

<function __main__.plot_reward(c=0.25, c1=0.5, max_descartados=10)>

In [10]:
def plot_reward(c=0.25, c2=0.5, divisiones=10):
    len_cola = 250
    porcentaje_paquetes = 0.02
    paquetes = int(porcentaje_paquetes*len_cola)

    def local_reward(action: Acciones,
                     ocu_actual: float,
                     c: float,
                     c2: float) -> float:

        if action == Acciones.DENEGAR:
            return reward(descartados=paquetes, action=action, ocu_actual=ocu_actual, ocu_ant=0.0,c=c, c2=c2,c3=0.0)

        ocu: float = ocu_actual+porcentaje_paquetes
        if ocu <= 1.0:
            return reward(descartados=0, action=action, ocu_actual=ocu_actual, ocu_ant=0.0, c=c, c2=c2, c3=0.0)
        else:
            descartados = int((ocu - 1.0) * len_cola)
            return reward(descartados=descartados, action=action, ocu_actual=ocu_actual, ocu_ant=0.0, c=c, c2=c2, c3=0.0)

    ocupacion_range = np.linspace(0, 1, divisiones)

    rewards_permitir: list[float] = [local_reward(
        Acciones.PERMITIR, o, c, c2) for o in ocupacion_range]
    rewards_bloquear: list[float] = [local_reward(
        Acciones.DENEGAR, o, c, c2) for o in ocupacion_range]

    plt.figure(figsize=(8, 5))
    plt.plot(ocupacion_range, rewards_permitir,
             label="Acción: PERMITIR", marker="o", linestyle="--")
    plt.plot(ocupacion_range, rewards_bloquear,
             label="Acción: DENEGAR", marker="s", linestyle="-")
    plt.axhline(0, color="black", linewidth=1, linestyle="dotted")

    plt.xlabel("Ocupación")
    plt.ylabel("Recompensa")
    plt.title("Impacto de c y c1 en la recompensa")
    plt.legend()
    plt.grid(True)


# Crear widgets interactivos
interact(plot_reward,
         c=(0, 4, 0.05),
         c2=(0, 4, 0.05),
         divisiones=(10, 200, 1),
         )

plt.show()

interactive(children=(FloatSlider(value=0.25, description='c', max=4.0, step=0.05), FloatSlider(value=0.5, des…

In [11]:
def local_reward(action: Acciones, ocu_actual: float, ocu_ant: float, c: float, c2: float, c3: float) -> float:
    len_cola = 250
    porcentaje_paquetes = 0.02
    paquetes = int(porcentaje_paquetes * len_cola)

    if action == Acciones.DENEGAR:
        return reward(descartados=paquetes, action=action, ocu_actual=ocu_actual, ocu_ant=ocu_ant, c=c, c2=c2, c3=c3)

    ocu: float = ocu_actual + porcentaje_paquetes
    if ocu <= 1.0:
        return reward(descartados=0, action=action, ocu_actual=ocu_actual, ocu_ant=ocu_ant, c=c, c2=c2, c3=c3)
    else:
        descartados = int((ocu - 1.0) * len_cola)
        return reward(descartados=descartados, action=action, ocu_actual=ocu_actual, ocu_ant=ocu_ant, c=c, c2=c2, c3=c3)

# Función para graficar en 3D
def plot_reward(c=0.25, c2=0.5, c3=1.0, divisiones=10):
    ocupacion_range = np.linspace(0, 1, divisiones)
    ocupacion_ant_range = np.linspace(0, 1, divisiones)  # Ocupación anterior

    X, Y = np.meshgrid(ocupacion_ant_range, ocupacion_range)
    Z_permitir = np.array([[local_reward(Acciones.PERMITIR, ocu, ocu_ant, c, c2, c3) for ocu_ant in ocupacion_ant_range] for ocu in ocupacion_range])
    Z_bloquear = np.array([[local_reward(Acciones.DENEGAR, ocu, ocu_ant, c, c2, c3) for ocu_ant in ocupacion_ant_range] for ocu in ocupacion_range])

    fig = plt.figure(figsize=(12, 6))
    
    # Gráfico 3D para acción PERMITIR
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.plot_surface(X, Y, Z_permitir, cmap="viridis", edgecolor="k", alpha=0.8)
    ax1.set_title("Recompensa - Acción PERMITIR")
    ax1.set_xlabel("Ocupación Anterior")
    ax1.set_ylabel("Ocupación Actual")
    ax1.set_zlabel("Recompensa")

    # Gráfico 3D para acción DENEGAR
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.plot_surface(X, Y, Z_bloquear, cmap="plasma", edgecolor="k", alpha=0.8)
    ax2.set_title("Recompensa - Acción DENEGAR")
    ax2.set_xlabel("Ocupación Anterior")
    ax2.set_ylabel("Ocupación Actual")
    ax2.set_zlabel("Recompensa")

   

# Crear widgets interactivos
interact(plot_reward,
         c=(0, 4, 0.05),
         c2=(0, 4, 0.05),
         c3=(0, 4, 0.05),
         divisiones=(10, 200, 1))
plt.show()

interactive(children=(FloatSlider(value=0.25, description='c', max=4.0, step=0.05), FloatSlider(value=0.5, des…

In [12]:
def local_reward(action: Acciones, ocu_actual: float, ocu_ant: float, c: float, c2: float, c3: float) -> float:
    len_cola = 250
    porcentaje_paquetes = 0.02
    paquetes = int(porcentaje_paquetes * len_cola)

    if action == Acciones.DENEGAR:
        return reward(descartados=paquetes, action=action, ocu_actual=ocu_actual, ocu_ant=ocu_ant, c=c, c2=c2, c3=c3)

    ocu: float = ocu_actual + porcentaje_paquetes
    if ocu <= 1.0:
        return reward(descartados=0, action=action, ocu_actual=ocu_actual, ocu_ant=ocu_ant, c=c, c2=c2, c3=c3)
    else:
        descartados = int((ocu - 1.0) * len_cola)
        return reward(descartados=descartados, action=action, ocu_actual=ocu_actual, ocu_ant=ocu_ant, c=c, c2=c2, c3=c3)

# Función para graficar ambas recompensas en un solo gráfico 3D
def plot_reward(c=0.25, c2=0.5, c3=1.0, divisiones=10):
    ocupacion_range = np.linspace(0, 1, divisiones)
    ocupacion_ant_range = np.linspace(0, 1, divisiones)

    X, Y = np.meshgrid(ocupacion_ant_range, ocupacion_range)
    Z_permitir = np.array([[local_reward(Acciones.PERMITIR, ocu, ocu_ant, c, c2, c3) for ocu_ant in ocupacion_ant_range] for ocu in ocupacion_range])
    Z_bloquear = np.array([[local_reward(Acciones.DENEGAR, ocu, ocu_ant, c, c2, c3) for ocu_ant in ocupacion_ant_range] for ocu in ocupacion_range])

    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')

    # Superficie de acción PERMITIR
    ax.plot_surface(X, Y, Z_permitir, cmap="viridis", edgecolor="k", alpha=0.7, label="PERMITIR")

    # Superficie de acción DENEGAR
    ax.plot_surface(X, Y, Z_bloquear, cmap="plasma", edgecolor="k", alpha=0.7, label="DENEGAR")

    ax.set_title("Recompensa en función de Ocupación y Ocupación Anterior")
    ax.set_xlabel("Ocupación Anterior")
    ax.set_ylabel("Ocupación Actual")
    ax.set_zlabel("Recompensa")

    

# Crear widgets interactivos
interact(plot_reward,
         c=(0, 4, 0.05),
         c2=(0, 4, 0.05),
         c3=(0, 4, 0.05),
         divisiones=(10, 200, 1))
plt.show()

interactive(children=(FloatSlider(value=0.25, description='c', max=4.0, step=0.05), FloatSlider(value=0.5, des…